In [1]:
# === Step 1: Install & Imports ===
!pip -q install Sastrawi emoji

import re
import pandas as pd
from pathlib import Path

# Emoji handling
import emoji

# Sastrawi (Bahasa Indonesia)
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Inisialisasi stemmer & stopwords
stemmer = StemmerFactory().create_stemmer()
stopwords_id = set(StopWordRemoverFactory().get_stop_words())

# (Opsional) tambahkan stopwords domain KAI
stopwords_domain = {
    "kai","kaiaccess","access","kereta","krl","min","admin",
    "tolong","mohon","terima","kasih","terimakasih"
}
stopwords_all = stopwords_id.union(stopwords_domain)

pd.set_option("display.max_colwidth", 200)


In [2]:
# === Step 2: Load File ===
INPUT_FILE = "/Users/ahmadzaki/Downloads/Skripsi Zaki/CodeSkripsi/01.Cleaning/kai_final_merged_25okt2025.csv"

use_dtypes = {"id_str":"string","user_id_str":"string","conversation_id_str":"string"}
df = pd.read_csv(INPUT_FILE, dtype=use_dtypes, low_memory=False)

# Pastikan kolom teks ada
assert "full_text" in df.columns, "Kolom 'full_text' tidak ditemukan pada CSV."

print("Loaded:", INPUT_FILE)
print("Rows:", len(df))
df[["created_at","id_str","user_id_str","conversation_id_str","full_text"]].head(5)

Loaded: /Users/ahmadzaki/Downloads/Skripsi Zaki/CodeSkripsi/01.Cleaning/kai_final_merged_25okt2025.csv
Rows: 1743


,created_at,id_str,user_id_str,conversation_id_str,full_text
0,Fri Dec 30 09:57:34 +0000 2022,1608764220451213313,230609460,1608764220451213313,udah beli tiket di aplikasi kaiaccess dan saat pembelian dan pembayaran mengikutsertakan asuransi perjalanan Sompo Insurance Indonesia tetapi belum ada imel masuk dari asuransi utk polisnya bagaim...
1,Fri Dec 30 01:11:02 +0000 2022,1608631716184489986,1158718283870138368,1608631716184489986,@KAI121 kenapa gabisa pake ovo buat pembayaran kaiaccess min ?
2,Thu Dec 29 00:47:15 +0000 2022,1608263340564058116,1250731212471074816,1608263340564058116,@KAI121 min saya mau ubah no. Hp yang ada di kaiaccess. Mohon bantuannya min
3,Wed Dec 28 22:55:35 +0000 2022,1608235238366535680,120352418,1608227080503955458,Waktunya menikmati pemandangan sepanjang perjalanan ini ditemani segelas teh premium nikmat dan segar. Pesannya tadi lewat #KAIAccess dan bayarnya pakai gopay. Mudah dan praktis. @KAI121 https://t...
4,Wed Dec 28 10:38:04 +0000 2022,1608049638111850496,967360502652223489,1608049638111850496,@KAI121 kenapa KAIAccess dari kemaren susah di acces loading terus


In [3]:
# === Step 3: Case Folding ===
work = df.copy()
work["text_casefold"] = work["full_text"].fillna("").str.lower()

work[["full_text","text_casefold"]].head(10)


,full_text,text_casefold
0,udah beli tiket di aplikasi kaiaccess dan saat pembelian dan pembayaran mengikutsertakan asuransi perjalanan Sompo Insurance Indonesia tetapi belum ada imel masuk dari asuransi utk polisnya bagaim...,udah beli tiket di aplikasi kaiaccess dan saat pembelian dan pembayaran mengikutsertakan asuransi perjalanan sompo insurance indonesia tetapi belum ada imel masuk dari asuransi utk polisnya bagaim...
1,@KAI121 kenapa gabisa pake ovo buat pembayaran kaiaccess min ?,@kai121 kenapa gabisa pake ovo buat pembayaran kaiaccess min ?
2,@KAI121 min saya mau ubah no. Hp yang ada di kaiaccess. Mohon bantuannya min,@kai121 min saya mau ubah no. hp yang ada di kaiaccess. mohon bantuannya min
3,Waktunya menikmati pemandangan sepanjang perjalanan ini ditemani segelas teh premium nikmat dan segar. Pesannya tadi lewat #KAIAccess dan bayarnya pakai gopay. Mudah dan praktis. @KAI121 https://t...,waktunya menikmati pemandangan sepanjang perjalanan ini ditemani segelas teh premium nikmat dan segar. pesannya tadi lewat #kaiaccess dan bayarnya pakai gopay. mudah dan praktis. @kai121 https://t...
4,@KAI121 kenapa KAIAccess dari kemaren susah di acces loading terus,@kai121 kenapa kaiaccess dari kemaren susah di acces loading terus
5,@KAI121 halo saya abis beli tiket kereta di aplikasi kaiaccess udah saya bayar saldo saya kepotong tapi tiketnya belum kebeli sedangkan batas waktu habis dan tiket ga terbit,@kai121 halo saya abis beli tiket kereta di aplikasi kaiaccess udah saya bayar saldo saya kepotong tapi tiketnya belum kebeli sedangkan batas waktu habis dan tiket ga terbit
6,@KAI121 min aplikasi kaiaccess lagi ngadat kah? berat banget skg ngeload datanya. giliran udah tampil sering banget gabisa diproses munculnya server bekerja terlalu keras. Kalian ada masalah apa? ...,@kai121 min aplikasi kaiaccess lagi ngadat kah? berat banget skg ngeload datanya. giliran udah tampil sering banget gabisa diproses munculnya server bekerja terlalu keras. kalian ada masalah apa? ...
7,ada yang tau jam buka loket stasiun sidoarjo beli tiket kereta bukanya jam berapa? mau beli tiket yg keberangkatan pagi soalnya takut belum buka beli nde kaiaccess ga punya saldo samsek :( Sdf!,ada yang tau jam buka loket stasiun sidoarjo beli tiket kereta bukanya jam berapa? mau beli tiket yg keberangkatan pagi soalnya takut belum buka beli nde kaiaccess ga punya saldo samsek :( sdf!
8,Halo @KAI121 apps nya lagi under constructions apa jaringan sinyal @myXLCare yg kurang baik di Cilacap? Bolak-balik-bolak matiin mode pesawat masuk App #KaiAccess ga bisa² mau pesen tiket aja mash...,halo @kai121 apps nya lagi under constructions apa jaringan sinyal @myxlcare yg kurang baik di cilacap? bolak-balik-bolak matiin mode pesawat masuk app #kaiaccess ga bisa² mau pesen tiket aja mash...
9,@KAI121 Hallo @KAI121 kenapaa app KAIaccess tidak bisa digunakan yaa?,@kai121 hallo @kai121 kenapaa app kaiaccess tidak bisa digunakan yaa?


In [4]:
# === Step 4: Cleaning ===
def clean_text(s: str) -> str:
    s = str(s)
    s = re.sub(r"http\S+|www\.\S+", " ", s)     # URL
    s = re.sub(r"@\w+", " ", s)                 # mention
    s = re.sub(r"#", " ", s)                    # hapus simbol '#' (biarkan kata hashtag)
    s = re.sub(r"[^\w\s]", " ", s)              # hapus tanda baca (non huruf/angka/underscore)
    s = re.sub(r"\s+", " ", s).strip()          # rapikan spasi
    return s

work["text_clean"] = work["text_casefold"].apply(clean_text)

work[["text_casefold","text_clean"]].head(10)


,text_casefold,text_clean
0,udah beli tiket di aplikasi kaiaccess dan saat pembelian dan pembayaran mengikutsertakan asuransi perjalanan sompo insurance indonesia tetapi belum ada imel masuk dari asuransi utk polisnya bagaim...,udah beli tiket di aplikasi kaiaccess dan saat pembelian dan pembayaran mengikutsertakan asuransi perjalanan sompo insurance indonesia tetapi belum ada imel masuk dari asuransi utk polisnya bagaim...
1,@kai121 kenapa gabisa pake ovo buat pembayaran kaiaccess min ?,kenapa gabisa pake ovo buat pembayaran kaiaccess min
2,@kai121 min saya mau ubah no. hp yang ada di kaiaccess. mohon bantuannya min,min saya mau ubah no hp yang ada di kaiaccess mohon bantuannya min
3,waktunya menikmati pemandangan sepanjang perjalanan ini ditemani segelas teh premium nikmat dan segar. pesannya tadi lewat #kaiaccess dan bayarnya pakai gopay. mudah dan praktis. @kai121 https://t...,waktunya menikmati pemandangan sepanjang perjalanan ini ditemani segelas teh premium nikmat dan segar pesannya tadi lewat kaiaccess dan bayarnya pakai gopay mudah dan praktis
4,@kai121 kenapa kaiaccess dari kemaren susah di acces loading terus,kenapa kaiaccess dari kemaren susah di acces loading terus
5,@kai121 halo saya abis beli tiket kereta di aplikasi kaiaccess udah saya bayar saldo saya kepotong tapi tiketnya belum kebeli sedangkan batas waktu habis dan tiket ga terbit,halo saya abis beli tiket kereta di aplikasi kaiaccess udah saya bayar saldo saya kepotong tapi tiketnya belum kebeli sedangkan batas waktu habis dan tiket ga terbit
6,@kai121 min aplikasi kaiaccess lagi ngadat kah? berat banget skg ngeload datanya. giliran udah tampil sering banget gabisa diproses munculnya server bekerja terlalu keras. kalian ada masalah apa? ...,min aplikasi kaiaccess lagi ngadat kah berat banget skg ngeload datanya giliran udah tampil sering banget gabisa diproses munculnya server bekerja terlalu keras kalian ada masalah apa tolong seger...
7,ada yang tau jam buka loket stasiun sidoarjo beli tiket kereta bukanya jam berapa? mau beli tiket yg keberangkatan pagi soalnya takut belum buka beli nde kaiaccess ga punya saldo samsek :( sdf!,ada yang tau jam buka loket stasiun sidoarjo beli tiket kereta bukanya jam berapa mau beli tiket yg keberangkatan pagi soalnya takut belum buka beli nde kaiaccess ga punya saldo samsek sdf
8,halo @kai121 apps nya lagi under constructions apa jaringan sinyal @myxlcare yg kurang baik di cilacap? bolak-balik-bolak matiin mode pesawat masuk app #kaiaccess ga bisa² mau pesen tiket aja mash...,halo apps nya lagi under constructions apa jaringan sinyal yg kurang baik di cilacap bolak balik bolak matiin mode pesawat masuk app kaiaccess ga bisa² mau pesen tiket aja masha alloh sabar sabar
9,@kai121 hallo @kai121 kenapaa app kaiaccess tidak bisa digunakan yaa?,hallo kenapaa app kaiaccess tidak bisa digunakan yaa


In [5]:
# === Step 5: Tokenizing ===
def tokenize(s: str):
    return str(s).split()

work["tokens"] = work["text_clean"].apply(tokenize)

work[["text_clean","tokens"]].head(10)


,text_clean,tokens
0,udah beli tiket di aplikasi kaiaccess dan saat pembelian dan pembayaran mengikutsertakan asuransi perjalanan sompo insurance indonesia tetapi belum ada imel masuk dari asuransi utk polisnya bagaim...,"[udah, beli, tiket, di, aplikasi, kaiaccess, dan, saat, pembelian, dan, pembayaran, mengikutsertakan, asuransi, perjalanan, sompo, insurance, indonesia, tetapi, belum, ada, imel, masuk, dari, asur..."
1,kenapa gabisa pake ovo buat pembayaran kaiaccess min,"[kenapa, gabisa, pake, ovo, buat, pembayaran, kaiaccess, min]"
2,min saya mau ubah no hp yang ada di kaiaccess mohon bantuannya min,"[min, saya, mau, ubah, no, hp, yang, ada, di, kaiaccess, mohon, bantuannya, min]"
3,waktunya menikmati pemandangan sepanjang perjalanan ini ditemani segelas teh premium nikmat dan segar pesannya tadi lewat kaiaccess dan bayarnya pakai gopay mudah dan praktis,"[waktunya, menikmati, pemandangan, sepanjang, perjalanan, ini, ditemani, segelas, teh, premium, nikmat, dan, segar, pesannya, tadi, lewat, kaiaccess, dan, bayarnya, pakai, gopay, mudah, dan, praktis]"
4,kenapa kaiaccess dari kemaren susah di acces loading terus,"[kenapa, kaiaccess, dari, kemaren, susah, di, acces, loading, terus]"
5,halo saya abis beli tiket kereta di aplikasi kaiaccess udah saya bayar saldo saya kepotong tapi tiketnya belum kebeli sedangkan batas waktu habis dan tiket ga terbit,"[halo, saya, abis, beli, tiket, kereta, di, aplikasi, kaiaccess, udah, saya, bayar, saldo, saya, kepotong, tapi, tiketnya, belum, kebeli, sedangkan, batas, waktu, habis, dan, tiket, ga, terbit]"
6,min aplikasi kaiaccess lagi ngadat kah berat banget skg ngeload datanya giliran udah tampil sering banget gabisa diproses munculnya server bekerja terlalu keras kalian ada masalah apa tolong seger...,"[min, aplikasi, kaiaccess, lagi, ngadat, kah, berat, banget, skg, ngeload, datanya, giliran, udah, tampil, sering, banget, gabisa, diproses, munculnya, server, bekerja, terlalu, keras, kalian, ada..."
7,ada yang tau jam buka loket stasiun sidoarjo beli tiket kereta bukanya jam berapa mau beli tiket yg keberangkatan pagi soalnya takut belum buka beli nde kaiaccess ga punya saldo samsek sdf,"[ada, yang, tau, jam, buka, loket, stasiun, sidoarjo, beli, tiket, kereta, bukanya, jam, berapa, mau, beli, tiket, yg, keberangkatan, pagi, soalnya, takut, belum, buka, beli, nde, kaiaccess, ga, p..."
8,halo apps nya lagi under constructions apa jaringan sinyal yg kurang baik di cilacap bolak balik bolak matiin mode pesawat masuk app kaiaccess ga bisa² mau pesen tiket aja masha alloh sabar sabar,"[halo, apps, nya, lagi, under, constructions, apa, jaringan, sinyal, yg, kurang, baik, di, cilacap, bolak, balik, bolak, matiin, mode, pesawat, masuk, app, kaiaccess, ga, bisa², mau, pesen, tiket,..."
9,hallo kenapaa app kaiaccess tidak bisa digunakan yaa,"[hallo, kenapaa, app, kaiaccess, tidak, bisa, digunakan, yaa]"


In [6]:
# # === Step 6: Stopword Removal ===
# work["tokens_nostop"] = work["tokens"].apply(
#     lambda toks: [t for t in toks if t not in stopwords_all]
# )

# work[["tokens","tokens_nostop"]].head(10)


In [7]:
# === Step 7: Stemming ===
def stem_tokens(tokens):
    return [stemmer.stem(t) for t in tokens]

work["tokens_stem"] = work["tokens"].apply(stem_tokens)
work["text_stemmed"] = work["tokens_stem"].apply(lambda ts: " ".join(ts))

work[["tokens","tokens_stem","text_stemmed"]].head(10)


,tokens,tokens_stem,text_stemmed
0,"[udah, beli, tiket, di, aplikasi, kaiaccess, dan, saat, pembelian, dan, pembayaran, mengikutsertakan, asuransi, perjalanan, sompo, insurance, indonesia, tetapi, belum, ada, imel, masuk, dari, asur...","[udah, beli, tiket, di, aplikasi, kaiaccess, dan, saat, beli, dan, bayar, mengikutsertakan, asuransi, jalan, sompo, insurance, indonesia, tetapi, belum, ada, imel, masuk, dari, asuransi, utk, poli...",udah beli tiket di aplikasi kaiaccess dan saat beli dan bayar mengikutsertakan asuransi jalan sompo insurance indonesia tetapi belum ada imel masuk dari asuransi utk polis bagaimana ini
1,"[kenapa, gabisa, pake, ovo, buat, pembayaran, kaiaccess, min]","[kenapa, gabisa, pake, ovo, buat, bayar, kaiaccess, min]",kenapa gabisa pake ovo buat bayar kaiaccess min
2,"[min, saya, mau, ubah, no, hp, yang, ada, di, kaiaccess, mohon, bantuannya, min]","[min, saya, mau, ubah, no, hp, yang, ada, di, kaiaccess, mohon, bantu, min]",min saya mau ubah no hp yang ada di kaiaccess mohon bantu min
3,"[waktunya, menikmati, pemandangan, sepanjang, perjalanan, ini, ditemani, segelas, teh, premium, nikmat, dan, segar, pesannya, tadi, lewat, kaiaccess, dan, bayarnya, pakai, gopay, mudah, dan, praktis]","[waktu, nikmat, pandang, panjang, jalan, ini, tani, gelas, teh, premium, nikmat, dan, segar, pesan, tadi, lewat, kaiaccess, dan, bayar, pakai, gopay, mudah, dan, praktis]",waktu nikmat pandang panjang jalan ini tani gelas teh premium nikmat dan segar pesan tadi lewat kaiaccess dan bayar pakai gopay mudah dan praktis
4,"[kenapa, kaiaccess, dari, kemaren, susah, di, acces, loading, terus]","[kenapa, kaiaccess, dari, kemaren, susah, di, acces, loading, terus]",kenapa kaiaccess dari kemaren susah di acces loading terus
5,"[halo, saya, abis, beli, tiket, kereta, di, aplikasi, kaiaccess, udah, saya, bayar, saldo, saya, kepotong, tapi, tiketnya, belum, kebeli, sedangkan, batas, waktu, habis, dan, tiket, ga, terbit]","[halo, saya, abis, beli, tiket, kereta, di, aplikasi, kaiaccess, udah, saya, bayar, saldo, saya, potong, tapi, tiket, belum, kebel, sedang, batas, waktu, habis, dan, tiket, ga, terbit]",halo saya abis beli tiket kereta di aplikasi kaiaccess udah saya bayar saldo saya potong tapi tiket belum kebel sedang batas waktu habis dan tiket ga terbit
6,"[min, aplikasi, kaiaccess, lagi, ngadat, kah, berat, banget, skg, ngeload, datanya, giliran, udah, tampil, sering, banget, gabisa, diproses, munculnya, server, bekerja, terlalu, keras, kalian, ada...","[min, aplikasi, kaiaccess, lagi, ngadat, kah, berat, banget, skg, ngeload, data, gilir, udah, tampil, sering, banget, gabisa, proses, muncul, server, kerja, terlalu, keras, kalian, ada, masalah, a...",min aplikasi kaiaccess lagi ngadat kah berat banget skg ngeload data gilir udah tampil sering banget gabisa proses muncul server kerja terlalu keras kalian ada masalah apa tolong segera cari bantu...
7,"[ada, yang, tau, jam, buka, loket, stasiun, sidoarjo, beli, tiket, kereta, bukanya, jam, berapa, mau, beli, tiket, yg, keberangkatan, pagi, soalnya, takut, belum, buka, beli, nde, kaiaccess, ga, p...","[ada, yang, tau, jam, buka, loket, stasiun, sidoarjo, beli, tiket, kereta, buka, jam, berapa, mau, beli, tiket, yg, berangkat, pagi, soal, takut, belum, buka, beli, nde, kaiaccess, ga, punya, sald...",ada yang tau jam buka loket stasiun sidoarjo beli tiket kereta buka jam berapa mau beli tiket yg berangkat pagi soal takut belum buka beli nde kaiaccess ga punya saldo samsek sdf
8,"[halo, apps, nya, lagi, under, constructions, apa, jaringan, sinyal, yg, kurang, baik, di, cilacap, bolak, balik, bolak, matiin, mode, pesawat, masuk, app, kaiaccess, ga, bisa², mau, pesen, tiket,...","[halo, apps, nya, lagi, under, constructions, apa, jaring, sinyal, yg, kurang, baik, di, cilacap, bolak, balik, bolak, matiin, mode, pesawat, masuk, app, kaiaccess, ga, bisa, mau, sen, tiket, aja,...",halo apps nya lagi under constructions apa jaring sinyal yg kurang baik di cilacap bolak balik bolak matiin mode

In [8]:
# === Step 8: Emoji Handling (Drop vs Map) ===

def emoji_drop(s: str) -> str:
    return emoji.replace_emoji(str(s), replace="")

def emoji_map(s: str) -> str:
    # contoh hasil: "bagus 😊" -> "bagus :smiling_face_with_smiling_eyes:"
    return emoji.demojize(str(s), language="en")

work["emoji_drop_from_raw"] = work["full_text"].apply(emoji_drop)
work["emoji_map_from_raw"]  = work["full_text"].apply(emoji_map)

# Tampilkan contoh baris yang kemungkinan mengandung emoji
def has_emoji(text: str) -> bool:
    return emoji.emoji_count(str(text)) > 0

sample_emoji = work[work["full_text"].apply(has_emoji)].head(15)
sample_emoji[["full_text","emoji_drop_from_raw","emoji_map_from_raw"]]


,full_text,emoji_drop_from_raw,emoji_map_from_raw
61,Min @CommuterLine .. usulan saya kapan hari malah tambah parah nih server KAIAccess.. pesan lgsg TRK-NT gak bisa estafet baru bisa itupun nunggu sejam. Ini mslhnya apa IT SYSTEM? SERVER? Biaya MTC...,Min @CommuterLine .. usulan saya kapan hari malah tambah parah nih server KAIAccess.. pesan lgsg TRK-NT gak bisa estafet baru bisa itupun nunggu sejam. Ini mslhnya apa IT SYSTEM? SERVER? Biaya MTC...,Min @CommuterLine .. usulan saya kapan hari malah tambah parah nih server KAIAccess.. pesan lgsg TRK-NT gak bisa estafet baru bisa itupun nunggu sejam. Ini mslhnya apa IT SYSTEM? SERVER? Biaya MTC...
1272,Yuk segera beli tiket kereta api mu di Aplikasi Access by KAI bayar dengan MotionPay‼️ *s&amp;k berlaku #LiburanPraktis #MotionPay,Yuk segera beli tiket kereta api mu di Aplikasi Access by KAI bayar dengan MotionPay *s&amp;k berlaku #LiburanPraktis #MotionPay,Yuk segera beli tiket kereta api mu di Aplikasi Access by KAI bayar dengan MotionPay:double_exclamation_mark: *s&amp;k berlaku #LiburanPraktis #MotionPay
1715,Agak OOT tapi guys HATI-HATII bngt kalo pesen tiket via app 'Access by KAI' bayar via eWallet. Pas bayar notifikasinya mungkin FAIL. Somehow saldo kita jg belum ketarik. Kita otomatis akan beli ti...,Agak OOT tapi guys HATI-HATII bngt kalo pesen tiket via app 'Access by KAI' bayar via eWallet. Pas bayar notifikasinya mungkin FAIL. Somehow saldo kita jg belum ketarik. Kita otomatis akan beli ti...,Agak OOT tapi guys HATI-HATII bngt kalo pesen tiket via app 'Access by KAI' bayar via eWallet. Pas bayar notifikasinya mungkin FAIL. Somehow saldo kita jg belum ketarik. Kita otomatis akan beli ti...


In [9]:
# === Step 9: Save Stepwise Result (optional) ===
OUT_STEPWISE = "kai_preprocessed_stepwise_without_stopword.csv"
cols = [
    "id_str","created_at","user_id_str","conversation_id_str",
    "full_text",
    "text_casefold","text_clean","tokens","tokens_nostop",
    "tokens_stem","text_stemmed",
    "emoji_drop_from_raw","emoji_map_from_raw"
]
cols = [c for c in cols if c in work.columns]
work.to_csv(OUT_STEPWISE, index=False, encoding="utf-8-sig")
print("Saved:", OUT_STEPWISE, "| rows:", len(work))


Saved: kai_preprocessed_stepwise_without_stopword.csv | rows: 1743


In [10]:
# === Step 10: Label Schema & Annotation Columns ===
# Skema kelas (boleh kamu ubah, tapi konsisten sampai akhir)
LABEL_SCHEMA = {
    "pos": "positif",
    "neg": "negatif",
    "neu": "netral",
    # opsional: "unk": "tidak yakin"
}

# Pilih teks mana yang akan di-label (disarankan: hasil cleaning/stemming yang mau dipakai model)
# Kamu bisa ganti col_text di sini: "full_text" | "text_clean" | "text_stemmed"
col_text = "text_clean" if "text_clean" in work.columns else "full_text"

# Buat dataframe anotasi (copy agar aman)
ann = work[[
    "id_str", "created_at", "user_id_str", "conversation_id_str", col_text
]].copy()

# Namai kolom teksnya agar konsisten (TEXT)
ann = ann.rename(columns={col_text: "TEXT"})

# Kolom label kosong yang akan diisi manual
ann["LABEL"] = ""   # isilah dengan: pos | neg | neu  (sesuai LABEL_SCHEMA)

print("Annotation table siap. Baris:", len(ann))
ann.head(10)


Annotation table siap. Baris: 1743


,id_str,created_at,user_id_str,conversation_id_str,TEXT,LABEL
0,1608764220451213313,Fri Dec 30 09:57:34 +0000 2022,230609460,1608764220451213313,udah beli tiket di aplikasi kaiaccess dan saat pembelian dan pembayaran mengikutsertakan asuransi perjalanan sompo insurance indonesia tetapi belum ada imel masuk dari asuransi utk polisnya bagaim...,
1,1608631716184489986,Fri Dec 30 01:11:02 +0000 2022,1158718283870138368,1608631716184489986,kenapa gabisa pake ovo buat pembayaran kaiaccess min,
2,1608263340564058116,Thu Dec 29 00:47:15 +0000 2022,1250731212471074816,1608263340564058116,min saya mau ubah no hp yang ada di kaiaccess mohon bantuannya min,
3,1608235238366535680,Wed Dec 28 22:55:35 +0000 2022,120352418,1608227080503955458,waktunya menikmati pemandangan sepanjang perjalanan ini ditemani segelas teh premium nikmat dan segar pesannya tadi lewat kaiaccess dan bayarnya pakai gopay mudah dan praktis,
4,1608049638111850496,Wed Dec 28 10:38:04 +0000 2022,967360502652223489,1608049638111850496,kenapa kaiaccess dari kemaren susah di acces loading terus,
5,1607954768332492802,Wed Dec 28 04:21:05 +0000 2022,1010877687932440577,1607954768332492802,halo saya abis beli tiket kereta di aplikasi kaiaccess udah saya bayar saldo saya kepotong tapi tiketnya belum kebeli sedangkan batas waktu habis dan tiket ga terbit,
6,1607943108934914049,Wed Dec 28 03:34:45 +0000 2022,71733078,1607943108934914049,min aplikasi kaiaccess lagi ngadat kah berat banget skg ngeload datanya giliran udah tampil sering banget gabisa diproses munculnya server bekerja terlalu keras kalian ada masalah apa tolong seger...,
7,1607879185586061312,Tue Dec 27 23:20:45 +0000 2022,438072886,1607879185586061312,ada yang tau jam buka loket stasiun sidoarjo beli tiket kereta bukanya jam berapa mau beli tiket yg keberangkatan pagi soalnya takut belum buka beli nde kaiaccess ga punya saldo samsek sdf,
8,1607755461062950912,Tue Dec 27 15:09:07 +0000 2022,414602951,1607755461062950912,halo apps nya lagi under constructions apa jaringan sinyal yg kurang baik di cilacap bolak balik bolak matiin mode pesawat masuk app kaiaccess ga bisa² mau pesen tiket aja masha alloh sabar sabar,
9,1607603636284325890,Tue Dec 27 05:05:49 +0000 2022,359810843,1607603636284325890,hallo kenapaa app kaiaccess tidak bisa digunakan yaa,


In [11]:
# # === Step 11: Shuffle & Create Batches for Annotation ===
# import math

# SEED = 42
# BATCH_SIZE = 250  # ubah sesuai kenyamanan
# ann_shuf = ann.sample(frac=1.0, random_state=SEED).reset_index(drop=True)

# n_batches = math.ceil(len(ann_shuf) / BATCH_SIZE)
# print(f"Total rows: {len(ann_shuf)} | Batch size: {BATCH_SIZE} -> {n_batches} batch")

# # Simpan per batch
# batch_paths = []
# for i in range(n_batches):
#     lo = i * BATCH_SIZE
#     hi = min((i+1)*BATCH_SIZE, len(ann_shuf))
#     batch = ann_shuf.iloc[lo:hi].copy()
#     outp = f"kai_label_batch_{i+1:03d}.csv"
#     batch.to_csv(outp, index=False, encoding="utf-8-sig")
#     batch_paths.append(outp)

# batch_paths[:5]


In [12]:
# === Step 12: Single CSV for Annotation (Full) ===
ANN_FILE = "kai_unlabel_full_without_stopword.csv"
ann.to_csv(ANN_FILE, index=False, encoding="utf-8-sig")
print("Saved:", ANN_FILE, "| rows:", len(ann))


Saved: kai_unlabel_full_without_stopword.csv | rows: 1743


Pedoman Label:
- pos (positif): pujian, pengalaman baik, kepuasan, terima kasih (yang bermakna positif).
- neg (negatif): keluhan, error, gagal, marah, kecewa, sarkas yang jelas negatif.
- neu (netral): informasi, tanya murni (tanpa emosi), klarifikasi, permintaan bantuan tanpa opini.

Catatan:
- Fokus pada SENTIMEN terhadap layanan KAI Access (aplikasi/fitur/proses).
- Kalau kalimat campuran, nilai sentimen dominan.
- Abaikan balasan resmi KAI (sudah kita hapus sebelumnya).
- Isi kolom LABEL dengan: pos | neg | neu (huruf kecil, konsisten).
- Boleh isi catatan di kolom NOTE jika ragu (nanti bisa direview).


In [ ]:
# === Step 13: Load Labeled CSV(s) & Merge Back ===
import glob

# Pilih salah satu:

# 2) Atau kalau kamu pakai single file:
labeled_paths = ["kai_label_full_labeled_auto.csv"]

assert labeled_paths, "Tidak menemukan file label. Pastikan CSV label sudah disimpan."

# Load dan concat
labs = []
for p in labeled_paths:
    df_lab = pd.read_csv(p, dtype={"id_str":"string"})
    labs.append(df_lab)
labeled_all = pd.concat(labs, ignore_index=True)

# Validasi kolom wajib
for col in ["id_str","TEXT","LABEL"]:
    assert col in labeled_all.columns, f"Kolom {col} tidak ada di file label."

# Bersihkan label dan cek nilai unik
labeled_all["LABEL"] = labeled_all["LABEL"].astype(str).str.strip().str.lower()
print("Unique labels:", labeled_all["LABEL"].unique())

# Cek label yang tidak sah
valid_set = {"pos","neg","neu",""}  # "" untuk baris belum terisi
invalid = set(labeled_all["LABEL"].unique()) - valid_set
if invalid:
    print("Peringatan: ditemukan label tidak valid:", invalid)

# Merge ke dataframe kerja (work)
# Catatan: kolom TEXT pada label bersifat referensi; yang utama untuk join adalah id_str
work_labeled = work.merge(labeled_all[["id_str","LABEL","NOTE"]], on="id_str", how="left")

print("Rows after merge:", len(work_labeled))
work_labeled[["id_str","LABEL","NOTE","full_text"]].head(10)

Unique labels: ['neu' 'neg' 'pos']
Rows after merge: 1434


,id_str,LABEL,NOTE,full_text
0,1608764220451213313,neu,menyebut layanan tanpa opini,udah beli tiket di aplikasi kaiaccess dan saat pembelian dan pembayaran mengikutsertakan asuransi perjalanan Sompo Insurance Indonesia tetapi belum ada imel masuk dari asuransi utk polisnya bagaim...
1,1608631716184489986,neg,keluhan/tidak_bisa,@KAI121 kenapa gabisa pake ovo buat pembayaran kaiaccess min ?
2,1608263340564058116,neu,permintaan informasi/minta_bantuan,@KAI121 min saya mau ubah no. Hp yang ada di kaiaccess. Mohon bantuannya min
3,1608235238366535680,pos,pujian/mudah_praktis,Waktunya menikmati pemandangan sepanjang perjalanan ini ditemani segelas teh premium nikmat dan segar. Pesannya tadi lewat #KAIAccess dan bayarnya pakai gopay. Mudah dan praktis. @KAI121 https://t...
4,1608049638111850496,neg,keluhan/loading_lemot,@KAI121 kenapa KAIAccess dari kemaren susah di acces loading terus
5,1607954768332492802,neg,keluhan/saldo_kepotong,@KAI121 halo saya abis beli tiket kereta di aplikasi kaiaccess udah saya bayar saldo saya kepotong tapi tiketnya belum kebeli sedangkan batas waktu habis dan tiket ga terbit
6,1607943108934914049,neg,keluhan/tidak_bisa,@KAI121 min aplikasi kaiaccess lagi ngadat kah? berat banget skg ngeload datanya. giliran udah tampil sering banget gabisa diproses munculnya server bekerja terlalu keras. Kalian ada masalah apa? ...
7,1607879185586061312,pos,pujian/pujian,ada yang tau jam buka loket stasiun sidoarjo beli tiket kereta bukanya jam berapa? mau beli tiket yg keberangkatan pagi soalnya takut belum buka beli nde kaiaccess ga punya saldo samsek :( Sdf!
8,1607755461062950912,neu,menyebut layanan tanpa opini,Halo @KAI121 apps nya lagi under constructions apa jaringan sinyal @myXLCare yg kurang baik di Cilacap? Bolak-balik-bolak matiin mode pesawat masuk App #KaiAccess ga bisa² mau pesen tiket aja mash...
9,1607603636284325890,neg,keluhan/tidak_bisa,@KAI121 Hallo @KAI121 kenapaa app KAIaccess tidak bisa digunakan yaa?


In [ ]:
# === Step 14: Save Final Labeled Dataset ===
OUT_LABELED = "kai_labeled_final_test_sementara.csv"

# Pilih kolom penting untuk modeling
cols_keep = [
    "id_str","created_at","user_id_str","conversation_id_str",
    # Pilih salah satu representasi teks utama untuk model:
    # "full_text",
    "text_clean",            # contoh: pakai hasil cleaning
    # "text_stemmed",        # atau pakai hasil stemming
    "LABEL"
]
cols_keep = [c for c in cols_keep if c in work_labeled.columns]

final_labeled = work_labeled[cols_keep].dropna(subset=["LABEL"])
print("Distribusi label:")
print(final_labeled["LABEL"].value_counts())

final_labeled.to_csv(OUT_LABELED, index=False, encoding="utf-8-sig")
print("Saved:", OUT_LABELED, "| rows:", len(final_labeled))

Distribusi label:
LABEL
neu    862
neg    364
pos    208
Name: count, dtype: int64
Saved: kai_labeled_final_test_sementara.csv | rows: 1434
